In [14]:
# Note this only works for MSN's within the same world, otherwise the game fails to load any MSN
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")
old_msn = b"MU07_MS201"
new_msn = "WI01_MS101"

In [15]:
# Create a memdump in the room with the msn you want to replace
# search that memdump for locations where the MSN name is located
# In my testing you can use any one of the 4 locations that should appear, and it will work the same
# I would recommend creating a savestate in the cutscene/room before the fight loads, to make testing easier
dumppth = os.path.join("workspace", "riku.dump")
lib.parsingengine.load_memdump(dumppth)
addresses = lib.parsingengine.search_substr(old_msn)
addresses
# 0x0171b284'

['0x004f800b', '0x01c31228', '0x01c31308', '0x01d48e17']

In [16]:

starting_address = addresses[1][2:]

toByte = lambda b, n: hex(b)[2:].zfill(n)
padded_msn = new_msn[::-1].zfill(len(old_msn))[::-1]
bytes_msn = padded_msn.encode('utf-8')
words = []
codes = []
for i in range(0, len(bytes_msn), 4):
    byte = bytes_msn[i:i+4]
    def _getByte(n):
        if n >= len(bytes_msn):
            return 0
        return bytes_msn[n]
    word = {"addr": int(starting_address,16)+i, "value": bytearray([_getByte(i),_getByte(i+1),_getByte(i+2),_getByte(i+3)])}
    words.append(word)
for word in words:
    codes.append("{} {}".format(toByte(word["addr"], 8), ''.join([toByte(i,2) for i in word["value"]][::-1])))
codes = [code[0].replace("0","2") + code[1:] for code in codes]
codes

['21c31228 31304957', '21c3122c 31534d5f', '21c31230 00003130']

In [17]:
lib.cheatengine.apply_ram_code(codes)
lib.cheatengine.write_pnach(debug=True)

patch=1,EE,21C31228,extended,31304957

patch=1,EE,21C3122C,extended,31534D5F

patch=1,EE,21C31230,extended,00003130

